# Packages and variables

In [7]:
# Load packages
from sentence_transformers import models, losses, datasets, SentencesDataset
from sentence_transformers import SentenceTransformer, util, InputExample
import pandas as pd
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizer
import torch

In [2]:
# Specify variables
model_name = "bert-base-uncased"
train_batch_size = 20
max_seq_length = 250
num_epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = SentenceTransformer(model_name).to(device)

No sentence-transformers model found with name C:\Users\nisha/.cache\torch\sentence_transformers\bert-base-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\nisha/.cache\torch\sentence_transformers\bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mo

# Load and prepare dataset

In [3]:
from datasets import list_datasets, load_dataset
datasets_list = list_datasets()
if "multi_nli" in datasets_list:
    dataset = load_dataset("multi_nli")
    print(dataset)
else:
    print("Dataset not found")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [4]:
# Load dataset from huggingface
dataset = load_dataset("multi_nli")

# and make dataset as dataframe for easier usage
df = pd.DataFrame()
df["premise"] = dataset["train"]["premise"]
df["hypothesis"] = dataset["train"]["hypothesis"]
df["genre"] = dataset["train"]["genre"]
df["label"] = dataset["train"]["label"]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Dataset multi_nli downloaded and prepared to C:/Users/nisha/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# Create training dataloader
train_examples = []

# Each different hierarchy needs a different label
for i in df.iterrows():
    if i[1]["genre"] == "telephone":
        v = 0
    elif i[1]["genre"] == "government":
        v = 3
    elif i[1]["genre"] == "travel":
        v = 6
    elif i[1]["genre"] == "fiction":
        v = 9
    elif i[1]["genre"] == "slate":
        v = 12
    lab = int(i[1]["label"]) + v
    
    train_examples.append(InputExample(texts=[i[1]["premise"], i[1]["hypothesis"]], label=lab))

train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

# Train and save triplet model

In [6]:
# Specify loss function
train_loss = losses.BatchAllTripletLoss(model=model)

# Train
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          show_progress_bar=True,
          optimizer_params={'lr': 1e-05}
          )

# Save model
model.save("model_triplet_mnli")

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/19636 [00:00<?, ?it/s]

KeyboardInterrupt: 